In [1]:
#imports
import selenium as se
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from nltk import tokenize
import parameters
from time import sleep
from parsel import Selector
import csv
import pandas as pd
from selenium.webdriver.support.ui import Select
import pandas as pd

In [2]:
#open chrome
driver = webdriver.Chrome("/usr/bin/chromedriver")
driver.get('https://www.google.com/')
driver.maximize_window()

In [3]:
#starting webpage
url = 'https://uttarakhandtourism.gov.in/accommodation/type/homestay/?search_keyword=&location='
driver.get(url + str(3))

In [4]:
 #to scrape data from one page
def scrape_page(region):
    for i in range(1, 13):
        try:
            data = []
            
            info = driver.find_element_by_xpath("//div[@class = 'row']/div[@class = 'col-md-3 Homestay_colm mb-4 accomodation'][" + str(i) + "]")
            
            data.append(region)
            
            info = info.text.split('\n')
            
            data.extend(info[1:4])
            
            if info[4][:10] != 'No. of Bed':
                data.append('N/A')
                data.extend(info[4:-1])
            else:
                data.append(info[4][13:])
                data.extend(info[5:-1])
            
            #print(data)
            
            df.append(data)
        except:
            break

In [5]:
def get_region():
    
    location_value = driver.current_url.split('location=')[1][:-1]
    region = driver.find_element_by_xpath("//select[@id = 'location']/option[@value = " + str(location_value) + "]").text
    
    return region

In [6]:
def get_total_pages():
    pages = driver.find_element_by_class_name('pagination')
    total_pages = int(pages.text.split('\n')[-2])
    
    return total_pages

In [7]:
df = []

#selecting all regions one by one
for i in range(1, 49):
    select = Select(driver.find_element_by_id('location'))
    select.select_by_index(i)
    sleep(1)
    search = driver.find_element_by_id('btnContactUs')
    search.click()
    sleep(2)
    
    region = get_region()
    
    try:
        total_pages = get_total_pages()
        
        url_old = driver.current_url
        split = url_old.split('/homestay')
        for i in range(1, total_pages + 1):
            url_new = split[0] + '/homestay/page/' + str(i) + split[1]
            driver.get(url_new)
            
            #to scrape data from one page
            scrape_page(region)
            
        driver.get(url_old)
    except: 
        if driver.find_element_by_class_name('total').text[6] != '0':
            #to scrape data from one page
            scrape_page(region)
        continue

In [8]:
df

[['Almora',
  'SILVER',
  'Ajay Rastogi Homestay',
  'Mr Ajay Rastogi, S/o Late Satish Rastogi',
  '6',
  '9456597171',
  'N/A',
  'Majkhali Ranikhet Almora'],
 ['Almora',
  'SILVER',
  'Alok Sharma Homestay',
  'Mr. Alok Sharma, S/o Late Somdutt Sharma',
  '2',
  '8127333006',
  'N/A',
  'Village Malli Riyooni Majkhali Ranikhet Almora'],
 ['Almora',
  'SILVER',
  'Ambabari Home Stay',
  'Kesav Dutt Bhatt',
  '6',
  '7310919789',
  'arun_bhatt[at] yahoo[dot]in',
  'Vill Garhwali Post Kathpudiya Almora'],
 ['Almora',
  'GOLD',
  'Anand Ballabh Joshi Homestay',
  'Mr. Namesh Nandan Josh',
  '14',
  '7351581352, 8057207260',
  'joshinamesh@gmail.com',
  'PRANAM Nivas, Vill/Post- Kausani'],
 ['Almora',
  'SILVER',
  'Anand Singh Mehra Homestay',
  'Mr. Anand Singh Mehra, S/o Kishan Singh Mehra',
  '2',
  '9761310054',
  'N/A',
  'Village Gadholi Dinapani Almora'],
 ['Almora',
  'SILVER',
  'Anand Singh Mehra Homestay',
  'Anand Singh Mehra S/o Ram Singh Mehra',
  '10',
  '9456110201',
  'm

In [9]:
headers = ['Region', 'Category', 'Homestay name', 'Host name', 'No: of beds', 'Phone number', 'E-mail', 'Address']

In [10]:
data = pd.DataFrame(df, columns = headers)

In [11]:
data

,Region,Category,Homestay name,Host name,No: of beds,Phone number,E-mail,Address
0,Almora,SILVER,Ajay Rastogi Homestay,"Mr Ajay Rastogi, S/o Late Satish Rastogi",6,9456597171,N/A,Majkhali Ranikhet Almora
1,Almora,SILVER,Alok Sharma Homestay,"Mr. Alok Sharma, S/o Late Somdutt Sharma",2,8127333006,N/A,Village Malli Riyooni Majkhali Ranikhet Almora
2,Almora,SILVER,Ambabari Home Stay,Kesav Dutt Bhatt,6,7310919789,arun_bhatt[at] yahoo[dot]in,Vill Garhwali Post Kathpudiya Almora
3,Almora,GOLD,Anand Ballabh Joshi Homestay,Mr. Namesh Nandan Josh,14,"7351581352, 8057207260",joshinamesh@gmail.com,"PRANAM Nivas, Vill/Post- Kausani"
4,Almora,SILVER,Anand Singh Mehra Homestay,"Mr. Anand Singh Mehra, S/o Kishan Singh Mehra",2,9761310054,N/A,Village Gadholi Dinapani Almora
...,...,...,...,...,...,...,...,...
1491,Uttarkashi,SILVER,Trishul Athithi Awas Griha,"Virendra singh,",12,8057823397,N/A,Tekhla
1492,Uttarkashi,GOLD,Vandana Home Stay,"Tripan Singh Pawar,",12,8958015074,N/A,"Lata, Bhatwari"
1493,Uttarkashi,BRONZE,Vijay Home stay,"Vijay Singh Rana,",4,9012803426,N/A,Raithal
1494,Uttarkashi,---,Vinod Singh Rawat Homestay,"Mr Vinod Singh Rawat, S/o Mr Narayan Singh",6,7579400945,N/A,Village Balavat P/O Chiva Block Mori Uttarkashi


In [12]:
data.to_csv('Homestay_data.csv', index = False)